In [1]:
!git clone https://github.com/declare-lab/conv-emotion

Cloning into 'conv-emotion'...
remote: Enumerating objects: 1388, done.
remote: Counting objects: 100% (324/324), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 1388 (delta 156), reused 218 (delta 91), pack-reused 1064
Receiving objects: 100% (1388/1388), 751.31 MiB | 29.68 MiB/s, done.
Resolving deltas: 100% (700/700), done.
Checking out files: 100% (435/435), done.


In [ ]:
# print(len(list(videoSentence.values())[0]))
# print(len(list(videoLabels.values())[0]))

42
42


In [2]:
%cd /content/conv-emotion/TL-ERC/

/content/conv-emotion/TL-ERC


In [3]:
!python setup.py

Making iemocap dataset in the current folder conv-emotion\TL-ERC\datasets

In [4]:
!cp /content/drive/MyDrive/Depot_Mouhamed-20220228T100527Z-001/DialogueRNN/DialogueRNN/IEMOCAP_features/IEMOCAP_features_raw.pkl /content/conv-emotion/TL-ERC/datasets

In [5]:
!gdown --id 1nufbrBJ-LtcROv1MviCHFI7tQE3JnqQR

Downloading...
From: https://drive.google.com/uc?id=1nufbrBJ-LtcROv1MviCHFI7tQE3JnqQR
To: /content/conv-emotion/TL-ERC/iemocap.zip
100% 14.2M/14.2M [00:00<00:00, 110MB/s] 


In [ ]:
!unzip /content/conv-emotion/TL-ERC/iemocap.zip -d /content/conv-emotion/TL-ERC/datasets

store cornell weights in generative weights

In [7]:
%cd generative_weights
!gdown --id 1OXtnyJ5nDMmK75L9kEQvKPIyO0xzyeVC


/content/conv-emotion/TL-ERC/generative_weights
Downloading...
From: https://drive.google.com/uc?id=1OXtnyJ5nDMmK75L9kEQvKPIyO0xzyeVC
To: /content/conv-emotion/TL-ERC/generative_weights/cornell_weights.pkl
100% 42.0M/42.0M [00:00<00:00, 175MB/s]


In [8]:
%cd ..
%cd bert_model

/content/conv-emotion/TL-ERC
/content/conv-emotion/TL-ERC/bert_model


##Train the model

In [9]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 123 kB 5.2 MB/s 
     |████████████████████████████████| 132 kB 41.3 MB/s 
     |████████████████████████████████| 8.6 MB 48.1 MB/s 
     |████████████████████████████████| 79 kB 8.8 MB/s 
     |████████████████████████████████| 138 kB 45.5 MB/s 
     |████████████████████████████████| 127 kB 49.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [10]:
!pip install tensorboardX

     |████████████████████████████████| 125 kB 5.2 MB/s 


In [ ]:
#!python train.py --load_checkpoint=../generative_weights/cornell_weights.pkl --data=iemocap

In [12]:
from solver import *
from data_loader import get_loader
from configs import get_config
from util import Vocab
import os
import pickle


def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


config = get_config(parse = False, mode='train')
val_config = get_config(parse = False, mode='valid')
test_config = get_config(parse = False, mode='test')

# _RUNS = config.runs

# _best_test_loss, _best_test_f1_w, _best_test_f1_m, _best_epoch = [], [], [], []

# for run in range(_RUNS):

print(config)

# No. of videos to consider
training_data_len = int(config.training_percentage * \
    len(load_pickle(config.sentences_path)))


train_data_loader = get_loader(
    sentences=load_pickle(config.sentences_path)[:training_data_len],
    labels=load_pickle(config.label_path)[:training_data_len],
    conversation_length=load_pickle(config.conversation_length_path)[:training_data_len],
    sentence_length=load_pickle(config.sentence_length_path)[:training_data_len],
    batch_size=config.batch_size)

eval_data_loader = get_loader(
    sentences=load_pickle(val_config.sentences_path),
    labels=load_pickle(val_config.label_path),
    conversation_length=load_pickle(val_config.conversation_length_path),
    sentence_length=load_pickle(val_config.sentence_length_path),
    batch_size=val_config.eval_batch_size,
    shuffle=False)

test_data_loader = get_loader(
    sentences=load_pickle(test_config.sentences_path),
    labels=load_pickle(test_config.label_path),
    conversation_length=load_pickle(test_config.conversation_length_path),
    sentence_length=load_pickle(test_config.sentence_length_path),
    batch_size=test_config.eval_batch_size,
    shuffle=False)




# for testing
solver = Solver(config, train_data_loader,
                eval_data_loader, test_data_loader, is_train=True)


solver.build()

best_test_loss, best_test_f1_w, best_epoch = solver.train()

    # print(f"Current RUN: {run+1}")

    # print("\n\nBest test loss")
    # print(best_test_loss)
    # print("Best test f1 weighted")
    # print(best_test_f1_w)
    # print("Best epoch")
    # print(best_epoch)

    # _best_test_loss.append(best_test_loss)
    # _best_test_f1_w.append(best_test_f1_w)
    # _best_epoch.append(best_epoch)


# Print final
# print(f"\n\nAverage across runs:")

# print("Best epoch")
# print(_best_epoch)

# print("\n\nBest test loss")
# print(np.mean(np.array(_best_test_loss), axis=0))

# print("Overall test f1 weighted")
# print(np.array(_best_test_f1_w))

# print("Best test f1 weighted")
# print(np.mean(np.array(_best_test_f1_w), axis=0))

iemocap
iemocap
iemocap
Configurations
{'activation': 'Tanh',
 'batch_size': 2,
 'bidirectional': True,
 'checkpoint': None,
 'clip': 1.0,
 'context_size': 256,
 'conversation_length_path': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/train/conversation_length.pkl'),
 'data': 'iemocap',
 'data_dir': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/train'),
 'dataset_dir': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap'),
 'dropout': 0.0,
 'embedding_size': 300,
 'encoder_hidden_size': 768,
 'eval_batch_size': 2,
 'feedforward': 'FeedForward',
 'id2word_path': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/id2word.pkl'),
 'label_path': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/train/labels.pkl'),
 'learning_rate': 0.0001,
 'load_checkpoint': None,
 'minimum_improvement': 0.001,
 'mode': 'train',
 'model': 'bc_RNN',
 'n_epoch': 500,
 'num_bert_layers': 4,
 'num_classes': 6,
 'num_layers': 1,
 'optimizer': <class 'torch.optim.adam.Ada

100%|██████████| 231508/231508 [00:00<00:00, 2001234.38B/s]


Build Graph


100%|██████████| 407873900/407873900 [00:11<00:00, 36930478.35B/s]


Parameter initiailization
	context_encoder.rnn.weight_hh_l0
	encoder.embeddings.word_embeddings.weight True
	encoder.embeddings.position_embeddings.weight True
	encoder.embeddings.token_type_embeddings.weight True
	encoder.embeddings.LayerNorm.weight True
	encoder.embeddings.LayerNorm.bias True
	encoder.encoder.layer.0.attention.self.query.weight True
	encoder.encoder.layer.0.attention.self.query.bias True
	encoder.encoder.layer.0.attention.self.key.weight True
	encoder.encoder.layer.0.attention.self.key.bias True
	encoder.encoder.layer.0.attention.self.value.weight True
	encoder.encoder.layer.0.attention.self.value.bias True
	encoder.encoder.layer.0.attention.output.dense.weight True
	encoder.encoder.layer.0.attention.output.dense.bias True
	encoder.encoder.layer.0.attention.output.LayerNorm.weight True
	encoder.encoder.layer.0.attention.output.LayerNorm.bias True
	encoder.encoder.layer.0.intermediate.dense.weight True
	encoder.encoder.layer.0.intermediate.dense.bias True
	encoder.enc

...

Epoch 16 loss average: 0.241
train
              precision    recall  f1-score   support

           0     0.8636    0.8590    0.8613       376
           1     0.9485    0.9634    0.9558       764
           2     0.9129    0.9028    0.9078      1080
           3     0.9022    0.9119    0.9070       749
           4     0.9112    0.9077    0.9094       520
           5     0.9187    0.9157    0.9172      1210

    accuracy                         0.9144      4699
   macro avg     0.9095    0.9101    0.9098      4699
weighted avg     0.9144    0.9144    0.9144      4699

test
              precision    recall  f1-score   support

           0     0.4080    0.3542    0.3792       144
           1     0.6455    0.5796    0.6108       245
           2     0.4520    0.5521    0.4971       384
           3     0.5638    0.6235    0.5922       170
           4     0.5923    0.5686    0.5802       299
           5     0.5479    0.4803    0.5119       381

    accuracy                         

  2%|▉                                           | 1/48 [00:00<00:21,  2.14it/s]

Epoch: 17, iter 0: loss = 0.3484552204608917


100%|███████████████████████████████████████████| 48/48 [00:30<00:00,  1.55it/s]


Epoch 17 loss average: 0.208
train
              precision    recall  f1-score   support

           0     0.9122    0.8564    0.8834       376
           1     0.9506    0.9581    0.9544       764
           2     0.9114    0.9241    0.9177      1080
           3     0.9158    0.9146    0.9152       749
           4     0.9216    0.9269    0.9243       520
           5     0.9215    0.9215    0.9215      1210

    accuracy                         0.9223      4699
   macro avg     0.9222    0.9169    0.9194      4699
weighted avg     0.9223    0.9223    0.9222      4699

test
              precision    recall  f1-score   support

           0     0.4700    0.3264    0.3852       144
           1     0.6038    0.6531    0.6275       245
           2     0.4568    0.5781    0.5103       384
           3     0.5843    0.6118    0.5977       170
           4     0.6907    0.4482    0.5436       299
           5     0.5425    0.5696    0.5557       381

    accuracy                         

## Evaluation

In [13]:
validation_loss, w_valid_f1, valid_predictions = solver.evaluate(eval_data_loader, mode="valid")


In [14]:
labels = [e[1] for e in eval_data_loader]
Labels = [label for utt in labels for label in utt]
Labels = [e for k in Labels for e in k]

In [15]:
solver.print_metric(Labels, valid_predictions, "test")

test
              precision    recall  f1-score   support

           0     0.4571    0.2500    0.3232       128
           1     0.3358    0.6133    0.4340        75
           2     0.3833    0.4508    0.4143       244
           3     0.6194    0.4511    0.5220       184
           4     0.5455    0.4324    0.4824       222
           5     0.4723    0.5620    0.5133       258

    accuracy                         0.4608      1111
   macro avg     0.4689    0.4599    0.4482      1111
weighted avg     0.4808    0.4608    0.4596      1111



0.4595713379998748

## Inference

In [17]:
%cd ..

/content/conv-emotion/TL-ERC


In [18]:
import pickle

_, videoSpeakers, videoLabels, _, _, _, videoSentence, trainVid, testVid = pickle.load(open("/content/drive/MyDrive/Depot_Mouhamed-20220228T100527Z-001/DialogueRNN/DialogueRNN/IEMOCAP_features/IEMOCAP_features_raw.pkl", 'rb'), encoding='latin1')



In [20]:
from iemocap_preprocess import *

Loading SpaCy


In [21]:
conv = videoSentence['Ses01F_impro01']
conv

['Excuse me.',
 'Do you have your forms?',
 'Yeah.',
 'Let me see them.',
 'Is there a problem?',
 'Who told you to get in this line?',
 "Okay. But I didn't tell you to get in this line if you are filling out this particular form.",
 "Well what's the problem?  Let me change it.",
 'This form is a Z.X.four.',
 "You can't--  This is not the line for Z.X.four.  If you're going to fill out the Z.X.four, you need to have a different form of ID.",
 "What?  I'm getting an ID.  This is why I'm here.  My wallet was stolen.",
 'No. I need another set of ID to prove this is actually you.',
 'How am I supposed to get an ID without an ID?  How does a person get an ID in the first place?',
 "I don't know.  But I need an ID to pass this form along.  I can't just send it along without an ID.",
 "I'm here to get an ID.",
 'No.  I need another ID, a separate one.',
 'Like what?  Like a birth certificate?',
 "A birth certificate, a passport...a student ID; didn't you go to school?  Anything?",
 "Yes but 

In [22]:
len(conv)

26

In [23]:
labels = videoLabels['Ses01F_impro01']
labels

[2, 5, 2, 5, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5, 2, 3, 5]

In [24]:
len(labels)

26

26 répliques alors

In [25]:


parser = argparse.ArgumentParser()
    
# Load the dataset
iemocap = IEMOCAP()
iemocap.load_iemocap_data()

# Maximum valid length of sentence
# => SOS/EOS will surround sentence (EOS for source / SOS for target)
# => maximum length of tensor = max_sentence_length + 1
parser.add_argument('-s', '--max_sentence_length', type=int, default=30)

# Vocabulary
parser.add_argument('--max_vocab_size', type=int, default=20000)
parser.add_argument('--min_vocab_frequency', type=int, default=5)

args = parser.parse_args(args=[])

max_sent_len = args.max_sentence_length
max_conv_len = iemocap.max_conv_length
max_vocab_size = args.max_vocab_size
min_freq = args.min_vocab_frequency


conv_sentences = list([tokenize_conversation(conv)])
conversation_length = [min(len(conv), max_conv_len)]

conv_labels = [labels]
# fix labels as per conversation_length
for idx, conv_len in enumerate(conversation_length):
    conv_labels[idx]=conv_labels[idx][:conv_len]


sentences, sentence_length = pad_sentences(
    conv_sentences,
    max_sentence_length=max_sent_len,
    max_conversation_length=max_conv_len)

for sentence_len, label in zip(conversation_length, conv_labels):
    assert(sentence_len ==len(label))


In [28]:
len(sentences[0])

26

sentences[0] est le premier dialoque qui contient 26 répliques, tokenizés

In [30]:
infer_loader = get_loader(
    sentences,
    [[0 for s in conv] for conv in sentences ],
    conversation_length=conversation_length,
    sentence_length=sentence_length,
    batch_size=val_config.eval_batch_size,
    shuffle=False)

In [31]:
infer_loader

In [32]:
def inference(conversations, labels, conversation_length, sentence_length, type_ids, masks):
  # conversations: (batch_size) list of conversations
  #   conversation: list of sentences
  #   sentence: list of tokens
  # conversation_length: list of int
  # sentence_length: (batch_size) list of conversation list of sentence_lengths

  input_conversations = conversations

  # flatten input and target conversations
  input_sentences = [sent for conv in input_conversations for sent in conv]
  input_labels = [label for conv in labels for label in conv]
  input_sentence_length = [l for len_list in sentence_length for l in len_list]
  input_conversation_length = [l for l in conversation_length]
  input_masks = [mask for conv in masks for mask in conv]
  orig_input_labels = input_labels

  with torch.no_grad():
      # transfering the input to cuda
      input_sentences = to_var(torch.LongTensor(input_sentences))
      input_labels = to_var(torch.LongTensor(input_labels))
      input_sentence_length = to_var(torch.LongTensor(input_sentence_length))
      input_conversation_length = to_var(torch.LongTensor(input_conversation_length))
      input_masks = to_var(torch.LongTensor(input_masks))

  sentence_logits = solver.model(
      input_sentences,
      input_sentence_length,
      input_conversation_length,
      input_masks)

  present_predictions = list(np.argmax(sentence_logits.detach().cpu().numpy(), axis=1))
  print(present_predictions)

In [33]:
inference(*next(iter(infer_loader)))

[2, 5, 2, 5, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5, 2, 3, 5]


In [ ]:
import torch
import numpy as np

from util import to_var

In [ ]:
input = next(iter(eval_data_loader))
print(np.array(input[2],dtype=object ).shape)

(2,)


In [ ]:
sentences=load_pickle(val_config.sentences_path)

In [ ]:
l2i = {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger':6}
i2l = {v:k for k,v in l2i.items()}

In [ ]:
conv_id = 1
for replique,label in zip(sentences[conv_id],input[1][conv_id]):

  print(" ".join(replique))
  print(i2l[label]) 


In [ ]:
# Labels, sentences
labels=load_pickle(val_config.label_path)
sentences=load_pickle(val_config.sentences_path)

In [ ]:
labels = [[3,
 2,
 2,
 3]]